In [1]:
import json
import pandas as pd
from transformers import T5Tokenizer
import random

In [2]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

In [3]:
result = []

In [4]:
with open('squad_train.json', "r") as fh:
    source = json.load(fh)
    for article in source["data"]:
        for para in article["paragraphs"]:
            context = para["context"].replace("''", '" ').replace("``", '" ')
            context_tokens = tokenizer.tokenize(context.lower())
            
            for qa in para["qas"]:
                ques = qa["question"].replace("''", '" ').replace("``", '" ')
                ques_tokens = tokenizer.tokenize(ques.lower())

                for answer in qa["answers"]:
                    answer_text = answer["text"]
                    
                    start = answer["answer_start"]
                    end = start + len(answer_text)
                    
                    if len(context_tokens) < 120 and len(ques_tokens) < 30:
                        txt = "context : " + context.lower() + " query : " + ques.lower()
                        ans = answer_text.lower()
                        result.append([txt,ans])
                    else:
                        #s_context = context[:end+150]
                        i = end-1
                        while i < len(context) and context[i] != '.' and context[i] != '?':
                            i = i+1
                        s_context = context[:i+1]
                        
                        s_context_tokens = tokenizer.tokenize(s_context)
                        if len(s_context_tokens) < 120 and len(ques_tokens) < 30:
                            txt = "context : " + s_context.lower() + " query : " + ques.lower()
                            ans = answer_text.lower()
                            result.append([txt,ans])

In [5]:
random.shuffle(result)

In [6]:
len(result)

59942

In [7]:
result[:10]

[['context : puerto rico\'s constitution expressly forbids capital punishment, stating "the death penalty shall not exist", setting it apart from all u.s. states and territories other than michigan, which also has a constitutional prohibition (eleven other states and the district of columbia have abolished capital punishment through statutory law). however, capital punishment is still applicable to offenses committed in puerto rico, if they fall under the jurisdiction of the federal government, though federal death penalty prosecutions there have generated significant controversy. query : what territory\'s constitution says "the death penalty shall not exist"?',
  "puerto rico's"],
 ['context : a mailbox provider is an organization that provides services for hosting electronic mail domains with access to storage for mail boxes. it provides email servers to send, receive, accept, and store email for end users or other organizations. query : what is a mailbox provider? ',
  'an organizat

In [8]:
train_df = pd.DataFrame(result , columns = ['Text', 'Answer'])

In [9]:
train_df.to_csv('squad.csv', index=False)